CARGAR DIRECCIONES

In [3]:
import pandas as pd

# Definimos las rutas de los archivos
ruta_bch = 'REGISTROS_ORIGINALES/bch.csv'
ruta_bitcoin = 'REGISTROS_ORIGINALES/bitcoin.csv'
ruta_bnb = 'REGISTROS_ORIGINALES/bnb.csv'
ruta_etherium = 'REGISTROS_ORIGINALES/etherium.csv'
ruta_solana = 'REGISTROS_ORIGINALES/solana.csv'


AGREGAR COLUMNA DE TIPO DE DIVISA

In [4]:
def agregar_moneda(ruta_origen, nombre_moneda, nombre_archivo):
    df = pd.read_csv(ruta_origen)
    df['moneda'] = nombre_moneda
    ruta_destino = f'COLUMNA_NOMBRE/{nombre_archivo}'
    df.to_csv(ruta_destino, index=False)

agregar_moneda(ruta_bch, 'BCH', 'bch.csv')
agregar_moneda(ruta_bitcoin, 'Bitcoin', 'bitcoin.csv')
agregar_moneda(ruta_bnb, 'BNB', 'bnb.csv')
agregar_moneda(ruta_etherium, 'Ethereum', 'etherium.csv')
agregar_moneda(ruta_solana, 'Solana', 'solana.csv')

ESTANDARIZAR NOBRES DE COLUMNAS

In [5]:
nombres_columnas = {
    'timestamp': 'fecha_hora',
    'open': 'precio_apertura',
    'high': 'precio_maximo',
    'low': 'precio_minimo',
    'close': 'precio_cierre',
    'volume': 'volumen'
}

def estandarizar_nombres_columnas(ruta_origen, nombre_archivo):
    # Leer archivo usando pandas
    df = pd.read_csv(ruta_origen)
    
    # Renombrar columnas usando el diccionario
    df.rename(columns=nombres_columnas, inplace=True)
    
    # Ruta para guardar el archivo estandarizado
    ruta_destino = f'NOMBRES_ESTANDARIZADOS/{nombre_archivo}'
    
    # Guardar el archivo en la carpeta nueva
    df.to_csv(ruta_destino, index=False)

archivos_modificados = ['bch.csv', 'bitcoin.csv', 'bnb.csv', 'etherium.csv', 'solana.csv']
for archivo in archivos_modificados:
    ruta_origen = f'COLUMNA_NOMBRE/{archivo}'
    estandarizar_nombres_columnas(ruta_origen, archivo)



UNIR DOCUMENTOS

In [ ]:
archivos_estandarizados = ['bch.csv', 'bitcoin.csv', 'bnb.csv', 'etherium.csv', 'solana.csv']

# Leer todos los archivos CSV y concatenarlos
df_combinado = pd.concat([pd.read_csv(f'NOMBRES_ESTANDARIZADOS/{archivo}') 
                          for archivo in archivos_estandarizados], ignore_index=True)

# Guardar el DataFrame combinado en la carpeta UNIDOS
df_combinado.to_csv('ARCHIVOS_UNIDOS/archivos_combinados.csv', index=False)

ATOMIZAR EL ARCHIVO

In [16]:
# Leer el archivo combinado
df = pd.read_csv('ARCHIVOS_UNIDOS/archivos_combinados.csv')

# Eliminar "p. m." y "a. m." de la columna fecha_hora
df['fecha_hora'] = df['fecha_hora'].str.replace(r'\s(p\. m\.|a\. m\.)', '', regex=True)

# Convertir la columna fecha_hora a tipo datetime 
df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])

# Crear nuevas columnas separadas
df['Anio'] = df['fecha_hora'].dt.year
df['Mes'] = df['fecha_hora'].dt.month
df['Dia'] = df['fecha_hora'].dt.day
df['Hora'] = df['fecha_hora'].dt.strftime('%H:%M')

# Eliminar la columna fecha_hora
df.drop(columns=['fecha_hora'], inplace=True)

# Convertir todos los nombres de columnas a mayúsculas
df.columns = df.columns.str.upper()

# Guardar el DataFrame actualizado en un nuevo archivo
df.to_csv('ARCHIVO_ATOMICO/ARCHIVO_ATOMIZADO.csv', index=False)


Verificación de Valores Nulos

In [1]:
import pandas as pd

# Cargar el archivo ya combinado y atomizado
df = pd.read_csv('ARCHIVO_ATOMICO/ARCHIVO_ATOMIZADO.csv')

# Verificar valores nulos
nulos = df.isnull().sum()
print("Valores nulos por columna:\n", nulos)

# Verificar si hay filas duplicadas
duplicados = df.duplicated().sum()
print(f"\nNúmero de filas duplicadas: {duplicados}")

# Verificar tipos de datos
tipos = df.dtypes
print("\nTipos de datos por columna:\n", tipos)

# Validación de consistencia (ejemplo para columna de volumen)
if (df['VOLUMEN'] < 0).any():
    print("\n Hay valores negativos en la columna 'VOLUMEN'.")
else:
    print("\n Todos los valores en 'VOLUMEN' son válidos (mayores o iguales a cero).")


Valores nulos por columna:
 PRECIO_APERTURA    0
PRECIO_MAXIMO      0
PRECIO_MINIMO      0
PRECIO_CIERRE      0
VOLUMEN            0
MONEDA             0
ANIO               0
MES                0
DIA                0
HORA               0
dtype: int64

Número de filas duplicadas: 0

Tipos de datos por columna:
 PRECIO_APERTURA    float64
PRECIO_MAXIMO      float64
PRECIO_MINIMO      float64
PRECIO_CIERRE      float64
VOLUMEN            float64
MONEDA              object
ANIO                 int64
MES                  int64
DIA                  int64
HORA                object
dtype: object

 Todos los valores en 'VOLUMEN' son válidos (mayores o iguales a cero).


CAMBIO DE MES Y DIA

In [5]:
import pandas as pd
from datetime import datetime

# Cargar archivo
df = pd.read_csv('ARCHIVO_ATOMICO/ARCHIVO_ATOMIZADO.csv')

# Normalizar campos
df['MONEDA'] = df['MONEDA'].str.upper()
df['MES'] = df['MES'].astype(str).str.zfill(2)
df['DIA'] = df['DIA'].astype(str).str.zfill(2)

# Crear columna FECHA
df['FECHA'] = pd.to_datetime(df['ANIO'].astype(str) + '-' + df['MES'] + '-' + df['DIA'])
df['HORA'] = pd.to_datetime(df['HORA'], format="%H:%M").dt.time

# Mapas de traducción
dias_es = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miercoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sabado',
    'Sunday': 'Domingo'
}
meses_es = {
    'January': 'Enero',
    'February': 'Febrero',
    'March': 'Marzo',
    'April': 'Abril',
    'May': 'Mayo',
    'June': 'Junio',
    'July': 'Julio',
    'August': 'Agosto',
    'September': 'Septiembre',
    'October': 'Octubre',
    'November': 'Noviembre',
    'December': 'Diciembre'
}

# Agregar nombres en español
df['NOMBRE_DIA'] = df['FECHA'].dt.day_name().map(dias_es)
df['NOMBRE_MES'] = df['FECHA'].dt.month_name().map(meses_es)

# Eliminar columnas numéricas MES y DIA
df.drop(columns=['MES', 'DIA'], inplace=True)

# Guardar nuevo archivo
df.to_csv('ARCHIVO_ATOMICO/ARCHIVO_ATOMIZADO_2.csv', index=False)
print("Archivo actualizado con NOMBRE_DIA y NOMBRE_MES.")


Archivo actualizado con NOMBRE_DIA y NOMBRE_MES.
